# Table of Contents

[Import](#Import)  
[Functions](#Functions)  
[Data Processing](#Data-Processing)  
[Fix Strings](#Fix-Strings)  
[Dummy Variables](#Dummy-Variables)  

## Import  
[Table of Contents](#Table-of-Contents)

In [48]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize

from time import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

from nltk.tag import pos_tag
from textblob import TextBlob

import re
import copy

import pickle
import pyLDAvis.sklearn

In [3]:
!pwd

/Users/bauer/ds/metis/metisgh/Metis_Projects/Community_Survey/Code


In [28]:
df=pd.read_csv('../../../local_files/Projects/CommunityCensus/AVENCensus2016_data.csv')
# df=df.drop(['Unnamed: 0'],1)

In [29]:
df.head(2)

,timestamp,year,month,education,student,religion,religionOther,raceDesc,nationality,USRegion,...,notPartNaN,notPartNervous,notPartNoLocal,notPartNotInt,notPartRacism,notPartSocAnxiety,notPartNotAccDisab,notPartUnfriendToLGBTQ,notPartUnfriendToGreyDemi,notPartElitistNonAces
0,2016-10-23 18:57:01,1995.0,July,Some college (no degree yet),"Yes, in an undergraduate program",Agnostic,NaN,White,United States of America,Florida,...,0,0,1,1,0,0,0,0,1,0
1,2016-10-23 19:35:35,1998.0,September,Less than high school,"Yes, in high school",Roman Catholic,NaN,White American,United Kingdom,NaN,...,0,1,0,0,0,0,0,0,0,0


In [30]:
# with open("../Data_Prep/2016_likertDict.pkl", 'rb') as picklefile: 
#     likertDict = pickle.load(picklefile)

In [31]:
# list(df)

## Functions
[Table of Contents](#Table-of-Contents)

In [32]:
def top_words(model, feature_names, n_top_words):
    lst = []
    for topic_idx, topic in enumerate(model.components_):
#         lst.append(topic_idx)
        lst.append(" , ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    return lst

In [33]:
def nmfTopics(data, n_topics, n_top_words, n_features):

    # Use tf-idf features for NMF.
    print("Extracting tf-idf features for NMF...")
    tfidf_vectorizer = TfidfVectorizer(min_df=4, max_df=.25, 
#                                        max_features=int(len(data)),
                                       stop_words='english', ngram_range=(3, 3))
    t0 = time()
    tfidf = tfidf_vectorizer.fit_transform(data)
    print("done in %0.3fs." % (time() - t0))

    # Fit the NMF model
    print("Fitting the NMF model with tf-idf features, "
          "n_samples=%d and n_features=%d..."
          % (int(len(data)), int(len(data)/2)))
    t0 = time()
    nmf = NMF(n_components=n_topics, random_state=1,
              alpha=.1, l1_ratio=.3).fit(tfidf)
    print("done in %0.3fs." % (time() - t0))

    print("\nTopics in NMF model:")
    tfidf_feature_names = tfidf_vectorizer.get_feature_names()
    return top_words(nmf, tfidf_feature_names, n_top_words), nmf.fit_transform(tfidf)

In [34]:
def processing(data, col_num):
#     data = data.dropna(subset=[data.columns[col_num]])
    data = data.dropna(subset=[col_num])
    data = data.ix[:,col_num]
    data2 = data
    data.tolist()
    data = [x.lower() for x in data]
#     data2 = []
#     for i in range(len(data)):  
#         temp = []
#         for word in TextBlob(data[i]).words:
#             werd = stemmer.stem(word)
#             #print(' '.join(werd))
#             #print(werd)
#             temp.append(werd)
#             #" ".join(temp)
#         data2.append(temp)
#         data2[i] = " ".join(data2[i])
    return data, data2


## Data Processing
[Table of Contents](#Table-of-Contents)

In [35]:
long_answers = ["feedback", "outResponceLong", "discrimLong", "aceSupportLong",  "allySupportLong","questions", "stories" ]
for i in long_answers:
    df[i].replace([np.nan],[";"], inplace = True)

In [36]:
df['test'] =  df["outResponceLong"].map(str)+" "+  df["discrimLong"].map(str)+" "+  df["aceSupportLong"].map(str)\
            +" "+  df["allySupportLong"].map(str)  + " "+ df["questions"].map(str)+ " "+ df["stories"].map(str)#+ df["feedback"].map(str) +" "+
# df[df['test'].notnull()]
df['test'].replace(['; ; ; ; ; ;'],[np.nan], inplace = True)
df['test'].replace([';'],[np.nan], inplace = True)

In [37]:
df['support_long'] = df["aceSupportLong"].map(str)+" "+  df["allySupportLong"].map(str)
df['support_long'].replace(['   '],[np.nan], inplace = True)

In [38]:
df['outResponceLong'].replace([';'],[np.nan],inplace = True)

In [39]:
discrim, discrim_df = processing(df, 'test')

In [40]:
len(discrim)

6311

In [45]:
discrim

['i only tell people who seem familiar with or are in the lgbtqia community because they understand the confusion of trying to find yourself the most. they are also more ok with the sprectrum idea of identity, so i do not feel like they would invalidate me or barrage me with uncomfortable questions. the only "uncomfortable" response i\'ve gotten concerning my demisexuality/gray-a identity was he (a cis hetersexual male) assumed it was purely a relationship preference and not a separate sexuality. no ; ; ; ;',
 "i do not tell a lot of people about it, but the ones i did tell were mostly positive. one friend was always trying to put a label on me and joking a bit about it, before i was sure about my identity myself, which was annoying and kind of made me not want to identify as ace for a while. but i'm lucky because one of my best friends is ace too, so we can relate to each other and talk about it.  ; i think it's just nice to be able to talk about it, see that i am not the only one who

In [42]:

# for i in range(len(discrim)):
#     tagged_sent = pos_tag(discrim[i].split())
#     adjs = [word for word,pos in tagged_sent if pos == 'JJ']
#     for j in range(len(adjs)):
#         print('\n',adjs[j], '\n', TextBlob(adjs[j]).sentiment )

In [43]:
# adjs

In [44]:
tfidf_vectorizer = TfidfVectorizer(min_df=2, max_df=.5, 
#                                   max_features=int(len(data)),
                                    stop_words='english',strip_accents = 'unicode', lowercase = True, ngram_range=(3, 3))
tfidf = tfidf_vectorizer.fit_transform(discrim)
nmf = NMF(n_components=5, random_state=1,
              alpha=.1, l1_ratio=.3).fit(tfidf)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

In [49]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                               stop_words = 'english',
                               lowercase = True,
                               token_pattern = r'\b[a-zA-Z]{3,}\b',
                               max_df = 0.5,
                               min_df = 2,
                               ngram_range = (3,3))
dtm_tf = tf_vectorizer.fit_transform(discrim)
lda_tf = LatentDirichletAllocation(n_topics=10, random_state=0, n_jobs = -1)
lda_tf.fit(dtm_tf)
data = pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)
pyLDAvis.display(data)


/Users/bauer/anaconda3/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


In [50]:

tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                               stop_words = 'english',
                               lowercase = True,
                               token_pattern = r'\b[a-zA-Z]{3,}\b',
                               max_df = 0.5,
                               min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(discrim)
lda_tf = LatentDirichletAllocation(n_topics=10, random_state=0, n_jobs = -1)
lda_tf.fit(dtm_tf)
data = pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)
pyLDAvis.display(data)

/Users/bauer/anaconda3/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


In [51]:
# lda_tf.fit(dtm_tf)
tfidf_vectorizer.fit(discrim)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=None, min_df=2,
        ngram_range=(3, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents='unicode', sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [54]:
import pyLDAvis
import prepare from sklearn


ImportError: cannot import name 'prepare'

In [55]:
# data = pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)
data = pyLDAvis.sklearn.prepare(nmf, discrim, tfidf_vectorizer)
pyLDAvis.display(data)

AttributeError: 'list' object has no attribute 'sum'

In [ ]:
tokenize everything 
train['tokenizer'] = train['col'].apply(lambda x: word_tokenize(x))
countvetor(strip_accents = 'unicode', stop_word ='english', lowercase = True, max_df, min_df)


get shape of fit_transform

data = dyLDAvis.sklearn.prepare(model, )

In [58]:
topWords = top_words(nmf, tfidf_feature_names, 5)

In [57]:
sentences = ("The sun is shiny i like the sun","I have been exposed to sun")
vect = TfidfVectorizer(stop_words="english",lowercase=False)
mat = vect.fit_transform(sentences).toarray()
print(type(mat))
print(vect.vocabulary_)
words = ['']*len(vect.vocabulary_)
for k,v in vect.vocabulary_.items():
    words[v] = k
print(words)
print(vect.idf_)
# q = mat / vect.idf_
# sums = np.ones((q.shape[0], 1))
# lens = np.ones((q.shape[0], 1))
# for ix in range(q.shape[0]):
#     sums[ix] = np.sum(q[ix,:])
#     lens[ix] = len([x for x in sentences[ix].split() if x in vect.get_feature_names()]) #have to filter out stopwords
# sum_to_1 = q / sums
# tf = sum_to_1 * lens
# print(tf)

<class 'numpy.ndarray'>
{'like': 2, 'sun': 4, 'shiny': 3, 'The': 0, 'exposed': 1}
['The', 'exposed', 'like', 'shiny', 'sun']
[ 1.40546511  1.40546511  1.40546511  1.40546511  1.        ]


In [ ]:
clusters = [list(r).index(max(r)) for r in nmf.fit_transform(tfidf)]
clusters_Series = pd.DataFrame(clusters, index =discrim_df.index )
df_clus = df.join(clusters_Series)
# discrim_topic_df = df_clus
df_clus[0].value_counts()

In [59]:
topWords

['don tell people , tell people asexuality , generally don tell , tell people unless , tell people don',
 've told people , told people ve , told people responded , told people know , told people people',
 'told close friends , ve told close , close friends family , close friends ve , close friends responded',
 'haven met right , met right person , just haven met , tell just haven , told just haven',
 've told friends , told friends responded , friends responded positively , told friends ve , told friends sister']

In [ ]:
# sentences = ("The sun is shiny i like the sun","I have been exposed to sun")
# vect = TfidfVectorizer(stop_words="english",lowercase=False)
# mat = vect.fit_transform(sentences).toarray()
mat = tfidf_vectorizer.fit_transform(discrim)#.toarray()
print(mat.shape)
print(tfidf_vectorizer.get_feature_names().shape)
q = mat / tfidf_vectorizer.idf_
sums = np.ones((q.shape[0], 1))
lens = np.ones((q.shape[0], 1))
for ix in range(q.shape[0]):
    sums[ix] = np.sum(q[ix,:])
    lens[ix] = len([x for x in discrim[ix].split() if x in topWords]) #have to filter out stopwords
sum_to_1 = q / sums
tf = sum_to_1 * lens
# print(tf)

In [ ]:
list(tf)

In [ ]:
len(tfidf_vectorizer.get_feature_names())

In [ ]:
tfidf_vectorizer = TfidfVectorizer(min_df=4, max_df=.5, 
#                                   max_features=int(len(data)),
                                    stop_words='english', ngram_range=(3, 3))
tfidf = tfidf_vectorizer.fit_transform(discrim)
nmf = NMF(n_components=5, random_state=1,
              alpha=.1, l1_ratio=.3).fit(tfidf)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
top_words(nmf, tfidf_feature_names, 5)

In [ ]:
clusters = [list(r).index(max(r)) for r in nmf.fit_transform(tfidf)]
clusters_Series = pd.DataFrame(clusters, index =discrim_df.index )
df_clus = df.join(clusters_Series)
# discrim_topic_df = df_clus
df_clus[0].value_counts()

In [ ]:
discrim_topics, discrim_transform = nmfTopics(discrim,5,5,500)
discrim_topics

In [ ]:
clusters = [list(r).index(max(r)) for r in discrim_transform]
clusters_Series = pd.DataFrame(clusters, index =discrim_df.index )
df_clus = df.join(clusters_Series)
# discrim_topic_df = df_clus[df_clus['discrimLong'].notnull()]
discrim_topic_df = df_clus

In [ ]:
df_clus[0].value_counts()

In [ ]:
discrim_topics, discrim_transform = nmfTopics(discrim, 10,10,500)
discrim_topics

### Clustering

In [ ]:
# discrim_transform[1000:1025]

In [ ]:
# df_clus[df_clus[0]==0]['outResponceLong']#.value_counts()
df_clus[0].value_counts()

In [ ]:
for i in range(len(discrim_topics)):
    print('\n',discrim_topics[i], '\n', TextBlob(discrim_topics[i]).sentiment)

In [ ]:
for i in range(len(discrim_topics)):
    words = discrim_topics[i].split()
    for j in range(len(words)):
        print('\n',words[j], '\n', TextBlob(words[j]).sentiment)

In [ ]:
discrim_topic_df.columns

In [ ]:
discrim_topic_df['trans'].value_counts()

In [ ]:
discrim_topic_df[discrim_topic_df['trans']=='Yes'][0].value_counts()/len(discrim_topic_df[discrim_topic_df['trans']=='Yes'])

In [ ]:
discrim_topic_df[discrim_topic_df['trans']=='No'][0].value_counts()/len(discrim_topic_df[discrim_topic_df['trans']=='No'])

In [ ]:
discrim_topic_df[discrim_topic_df['trans']=='Unsure'][0].value_counts()/len(discrim_topic_df[discrim_topic_df['trans']=='Unsure'])

In [ ]:
discrim_topic_df[discrim_topic_df['gender']=='None of the above'][0].value_counts()/ len(discrim_topic_df[discrim_topic_df['gender']=='None of the above'])

In [ ]:
discrim_topic_df[discrim_topic_df['gender']=='Woman/female'][0].value_counts()/ len(discrim_topic_df[discrim_topic_df['gender']=='Woman/female'])

In [ ]:
discrim_topic_df[discrim_topic_df['gender']=='Man/male'][0].value_counts()/ len(discrim_topic_df[discrim_topic_df['gender']=='Man/male'])

In [ ]:
# discrim_topic_df[discrim_topic_df['year']<1996]['alcohol'].value_counts()#/len(discrim_topic_df[discrim_topic_df['year']<1996]['alcohol'])

In [ ]:
discrim_topic_df[discrim_topic_df['alcohol']<1][0].value_counts()#/ len(discrim_topic_df[discrim_topic_df['alcohol']<1])

In [ ]:
discrim_topic_df[discrim_topic_df['alcohol']>1][0].value_counts()#/ len(discrim_topic_df[discrim_topic_df['alcohol']>1])

topic_words = []
for r in model.components_:
    a = sorted([(v,i) for i,v in enumerate(r)],reverse=True)[0:7]
    topic_words.append([words[e[1]] for e in a])

## Fix Strings
[Table of Contents](#Table-of-Contents)

In [ ]:
stringing = 'hi (my, name, is) lots, of commas, here'
re.sub(r'\([^)]*\)', '', stringing)
#stringing.replace(r'([^0-9]', '')
#a = re.search(r'Release Date:</h4>[ ]+[0-9,a-z A-Z]+', page)
#b = re.search(r'>[ ][0-9,a-z A-Z]+', a.group(0))


## Dummy Variables
[Table of Contents](#Table-of-Contents)

In [ ]:
#list(df)

df.ix[2,'Do you identify with any of the following racial/ethnic categories?']

race = df['Do you identify with any of the following racial/ethnic categories?']
race = race.str.replace(r'\([^)]*\)', '')
race=race.str.replace(r'[ ]', '')
race = pd.Series([str.lower(str(i)).strip() for i in race])

race = race.str.get_dummies(sep=',') 

race.shape

race_cols = pd.DataFrame.sum(race)
fiveup = [race_cols.index[i] for i in range(len(race_cols.index)) if race_cols[i]>5.0]
fiveup = race[fiveup]
for i in fiveup:
    fiveup[i].fillna(0, inplace=True)
fiveup.columns = ['aboriginalAustralian','asianEastern','asianSoutheast','asianSouthern','asianWestern',
                  'blackAfrican','blackCaribbean','blackDiaspora', 'brown','hispanic','jewish','latinx','middleEastern',
                 'mixedRace','raceNoneSpecified','raceNo','northAfrican','northAmericanNative','pacificIslanderPoly',
                  'southCentAmericanNative','whiteOrEuropean']
fiveup.shape

fiveup_cols = pd.DataFrame.sum(fiveup)
fiveup_cols